In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/avacado-dataset/Avocado Dataset.csv


> # COM 526 P ANALYTICS & SYSTEMS OF BIG DATA PRACTICE – PROBLEM SET IV

> Name: Sindhiya
> Roll No: CS24M1005

# Question 1

> Suppose that the data for analysis includes the attribute age. The age values for the
> data tuples are (in increasing order) 13, 15, 16, 16, 19, 20, 20, 21, 22, 22, 25, 25, 25, 25,30, 33, 33, 35, 35, 35, 35, 36, 40, 45, 46, 52, 70.
>> (a) Use min-max normalization to transform the values of age to the range[0:1].
>>>(b) Use z-score normalization to transform the values of age.
>>>>(c) Use normalization by decimal scaling to transform the values of age such that the transformed value is less than 1.

a) min-max normalization

![image.png](attachment:906f6a26-91c8-447a-8062-30df0e918444.png)

In [ ]:
import numpy as np

# Age values
ages = np.array([13, 15, 16, 16, 19, 20, 20, 21, 22, 22, 25, 25, 25, 25,
                 30, 33, 33, 35, 35, 35, 35, 36, 40, 45, 46, 52, 70])

# Min-max normalization
min_age = np.min(ages)
max_age = np.max(ages)
min_max_normalized = (ages - min_age) / (max_age - min_age)
print("Min-Max Normalized Ages:", min_max_normalized)


Min-Max Normalized Ages: [0.         0.03508772 0.05263158 0.05263158 0.10526316 0.12280702
 0.12280702 0.14035088 0.15789474 0.15789474 0.21052632 0.21052632
 0.21052632 0.21052632 0.29824561 0.35087719 0.35087719 0.38596491
 0.38596491 0.38596491 0.38596491 0.40350877 0.47368421 0.56140351
 0.57894737 0.68421053 1.        ]


(b) Z-Score Normalization:

![image.png](attachment:2e6afa6a-71da-4741-9bb3-7940e27732ce.png)

In [ ]:
# Z-score normalization
mean_age = np.mean(ages)
std_age = np.std(ages)
z_score_normalized = (ages - mean_age) / std_age
print("Z-Score Normalized Ages:", z_score_normalized)


Z-Score Normalized Ages: [-1.33564599e+00 -1.17816807e+00 -1.09942912e+00 -1.09942912e+00
 -8.63212252e-01 -7.84473297e-01 -7.84473297e-01 -7.05734341e-01
 -6.26995386e-01 -6.26995386e-01 -3.90778520e-01 -3.90778520e-01
 -3.90778520e-01 -3.90778520e-01  2.91625761e-03  2.39133124e-01
  2.39133124e-01  3.96611035e-01  3.96611035e-01  3.96611035e-01
  3.96611035e-01  4.75349990e-01  7.90305812e-01  1.18400059e+00
  1.26273954e+00  1.73517328e+00  3.15247448e+00]


(c) Decimal Scaling Normalization:

![image.png](attachment:8c4b4394-6ca3-49b0-8ef6-6e43a700d117.png)

In [ ]:
# Decimal scaling normalization
max_abs_age = np.max(np.abs(ages))
j = np.ceil(np.log10(max_abs_age))
decimal_scaling_normalized = ages / (10**j)
print("Decimal Scaling Normalized Ages:", decimal_scaling_normalized)


Decimal Scaling Normalized Ages: [0.13 0.15 0.16 0.16 0.19 0.2  0.2  0.21 0.22 0.22 0.25 0.25 0.25 0.25
 0.3  0.33 0.33 0.35 0.35 0.35 0.35 0.36 0.4  0.45 0.46 0.52 0.7 ]


# Question 2

2. Operations on the Avocado Dataset

(a) Sorting and Smoothing Total Volume
We need to sort the "Total Volume" attribute and apply binning techniques.



In [ ]:
import pandas as pd

# Load the dataset
df = pd.read_csv('/kaggle/input/avacado-dataset/Avocado Dataset.csv')

# Sort by 'Total Volume'
df_sorted = df.sort_values(by='Total Volume')

# Number of bins
bins = 250
df_sorted['Total Volume Binned'] = pd.qcut(df_sorted['Total Volume'], bins, duplicates='drop')

# Bin Smoothing: Bin Means (explicitly set observed=False)
bin_means = df_sorted.groupby('Total Volume Binned', observed=False)['Total Volume'].mean()
df_sorted['Total Volume Bin Means'] = df_sorted['Total Volume Binned'].apply(lambda x: bin_means[x])

# Bin Smoothing: Bin Medians (explicitly set observed=False)
bin_medians = df_sorted.groupby('Total Volume Binned', observed=False)['Total Volume'].median()
df_sorted['Total Volume Bin Medians'] = df_sorted['Total Volume Binned'].apply(lambda x: bin_medians[x])

# Bin Smoothing: Bin Boundaries
def bin_boundaries(x, bin_labels):
    left, right = bin_labels.left, bin_labels.right
    return left if x - left < right - x else right

df_sorted['Total Volume Bin Boundaries'] = df_sorted.apply(
    lambda row: bin_boundaries(row['Total Volume'], row['Total Volume Binned']), axis=1)

# Display relevant columns
df_sorted[['Total Volume', 'Total Volume Bin Means', 'Total Volume Bin Medians', 'Total Volume Bin Boundaries']]


,Total Volume,Total Volume Bin Means,Total Volume Bin Medians,Total Volume Bin Boundaries
10381,84.56,7.357130e+02,774.20,8.455900e+01
9437,379.82,7.357130e+02,774.20,8.455900e+01
13189,385.55,7.357130e+02,774.20,8.455900e+01
11698,419.98,7.357130e+02,774.20,8.455900e+01
13193,472.82,7.357130e+02,774.20,8.455900e+01
...,...,...,...,...
5493,46324529.70,3.897352e+07,37352360.59,3.399446e+07
8353,47293921.60,3.897352e+07,37352360.59,3.399446e+07
5506,52288697.89,3.897352e+07,37352360.59,6.250565e+07
8366,61034457.10,3.897352e+07,37352360.59,6.250565e+07


(b) Monthly and Annual Sales
We need to group the data by month and year.

In [ ]:
# Convert 'Date' to datetime with dayfirst=True
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)

# Group by month
df['Month'] = df['Date'].dt.to_period('M')
monthly_sales = df.groupby('Month').agg({'Total Volume': 'sum'}).reset_index()

# Group by year
df['Year'] = df['Date'].dt.year
annual_sales = df.groupby('Year').agg({'Total Volume': 'sum'}).reset_index()

# Output monthly and annual sales
monthly_sales, annual_sales


(      Month  Total Volume
 0   2015-01  3.180852e+08
 1   2015-02  3.626376e+08
 2   2015-03  4.185831e+08
 3   2015-04  3.441444e+08
 4   2015-05  4.837840e+08
 5   2015-06  3.818267e+08
 6   2015-07  3.567019e+08
 7   2015-08  4.192448e+08
 8   2015-09  3.292188e+08
 9   2015-10  3.057761e+08
 10  2015-11  3.694400e+08
 11  2015-12  2.960259e+08
 12  2016-01  4.754058e+08
 13  2016-02  4.286458e+08
 14  2016-03  3.912124e+08
 15  2016-04  4.018328e+08
 16  2016-05  5.612305e+08
 17  2016-06  4.103282e+08
 18  2016-07  4.672753e+08
 19  2016-08  3.707970e+08
 20  2016-09  3.623033e+08
 21  2016-10  3.602018e+08
 22  2016-11  2.602093e+08
 23  2016-12  3.314478e+08
 24  2017-01  5.333330e+08
 25  2017-02  4.714672e+08
 26  2017-03  3.589034e+08
 27  2017-04  4.898145e+08
 28  2017-05  4.257305e+08
 29  2017-06  4.104107e+08
 30  2017-07  4.749037e+08
 31  2017-08  3.383330e+08
 32  2017-09  2.847795e+08
 33  2017-10  3.627780e+08
 34  2017-11  3.245193e+08
 35  2017-12  4.593329e+08
 

(c) Summarize Missing Values

In [ ]:
# Summarize missing values
missing_values = df.isnull().sum()
print("Missing Values Per Attribute:", missing_values)


Missing Values Per Attribute: Date             0
AveragePrice    28
Total Volume     0
4046             0
4225             0
4770             0
Total Bags       0
Small Bags       0
Large Bags       0
XLarge Bags      0
type             0
year             0
region           0
Month            0
Year             0
dtype: int64


(d) Populate Missing Values for 'Average Price' by 'Region'


In [ ]:
# Print column names to check for correct names
print(df.columns)

# Convert 'AveragePrice' to numeric to handle non-numeric data
df['AveragePrice'] = pd.to_numeric(df['AveragePrice'], errors='coerce')

# Ensure 'region' is spelled correctly by checking it in df.columns
if 'region' in df.columns:
    # Fill missing values in 'AveragePrice' based on 'region'
    df['AveragePrice'] = df.groupby('region')['AveragePrice'].transform(
        lambda grp: grp.fillna(grp.mean()))
else:
    print("'region' column not found. Check for any naming differences.")

    df['AveragePrice'] = df.groupby('Region')['AveragePrice'].transform(
    lambda grp: grp.fillna(grp.mean()))


Index(['Date', 'AveragePrice', 'Total Volume', '4046', '4225', '4770',
       'Total Bags', 'Small Bags', 'Large Bags', 'XLarge Bags', 'type', 'year',
       'region', 'Month', 'Year'],
      dtype='object')


(e) Discretize 'Date' into {Old, New, Recent}
We will categorize the years 2015, 2016 as "Old," 2017 as "New," and 2018 as "Recent."



In [ ]:
# Ensure 'Date' is in datetime format, specifying day comes first
df['Date'] = pd.to_datetime(df['Date'], errors='coerce', dayfirst=True)

# Extract the year from the 'Date' column
df['Year'] = df['Date'].dt.year

# Define the function to discretize the year
def discretize_date(year):
    if year in [2015, 2016]:
        return 'Old'
    elif year == 2017:
        return 'New'
    else:
        return 'Recent'

# Apply the function to create 'Date Category'
df['Date Category'] = df['Year'].apply(discretize_date)

# Display the 'Date' and 'Date Category' columns
df[['Date', 'Date Category']]


,Date,Date Category
0,2015-12-27,Old
1,2015-12-20,Old
2,2015-12-13,Old
3,2015-12-06,Old
4,2015-11-29,Old
...,...,...
18245,2018-01-28,Recent
18246,2018-01-21,Recent
18247,2018-01-14,Recent
18248,2018-01-07,Recent
